In [1]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


In [2]:
# Load the dataset
df = pd.read_csv('/content/cleaned_dataset_combined (3).csv')

# Convert string representation of list to actual list
df['tweet_tokens'] = df['tweet_tokens'].apply(ast.literal_eval)

# Define feature and target variables
X = df[['tweet_tokens']]
y = df['class']

In [3]:
# Define a function to join tokens into a single string
X['tweet_tokens'] = X['tweet_tokens'].apply(lambda x: ' '.join(x))

# Use TfidfVectorizer for the 'tweet_tokens' column
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X['tweet_tokens'])

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Handle class imbalance by converting labels to one-hot encoding
y_categorical = to_categorical(y_encoded)

<ipython-input-3-b52d4387a5c5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['tweet_tokens'] = X['tweet_tokens'].apply(lambda x: ' '.join(x))


In [4]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_categorical, test_size=0.2, random_state=42)

# Build the ANN model
model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_categorical.shape[1], activation='softmax'))

In [5]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with early stopping
model.fit(X_train.toarray(), y_train, epochs=50, batch_size=64, validation_data=(X_test.toarray(), y_test), callbacks=[early_stopping])

Epoch 1/50
310/310 [==============================] - 7s 11ms/step - loss: 0.4811 - accuracy: 0.8247 - val_loss: 0.3455 - val_accuracy: 0.8800
Epoch 2/50
310/310 [==============================] - 2s 7ms/step - loss: 0.2518 - accuracy: 0.9093 - val_loss: 0.3368 - val_accuracy: 0.8812
Epoch 3/50
310/310 [==============================] - 3s 8ms/step - loss: 0.1668 - accuracy: 0.9395 - val_loss: 0.3972 - val_accuracy: 0.8693
Epoch 4/50
310/310 [==============================] - 3s 8ms/step - loss: 0.1163 - accuracy: 0.9585 - val_loss: 0.4415 - val_accuracy: 0.8711
Epoch 5/50
310/310 [==============================] - 2s 7ms/step - loss: 0.0850 - accuracy: 0.9690 - val_loss: 0.5319 - val_accuracy: 0.8560


In [6]:

# Evaluate the model
loss, accuracy = model.evaluate(X_test.toarray(), y_test)
print(f'Test set accuracy: {accuracy:.2f}')

155/155 [==============================] - 1s 3ms/step - loss: 0.3368 - accuracy: 0.8812
Test set accuracy: 0.88
